In [21]:
import pandas as pd
import pyreadr
from rapidfuzz import process, fuzz

In [2]:
def load_rda(file_paths):

  dataframes = {}

  for file_path in file_paths:
      result = pyreadr.read_r(file_path)

      for key in result.keys():
          dataframes[key] = result[key]

  return dataframes

In [3]:
def missing_values(df):
  return(df.isnull().sum())

In [4]:
def outlier_values(df):
    outliers = pd.Series(dtype="float64")

    for col in df.select_dtypes(include=["number"]):  
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        mask = (df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))
        outliers[col] = mask.mean() * 100  
    return outliers

In [5]:
def clean_data(data_dict):
    cleaned_dict = {}
    for key, df in data_dict.items():
        df_cleaned = df.drop_duplicates()
        if 'RecordBeg' in df_cleaned.columns:
            df_cleaned.loc[:, 'RecordBeg'] = pd.to_datetime(df_cleaned['RecordBeg'], errors='coerce')
        if 'RecordEnd' in df_cleaned.columns:
            df_cleaned.loc[:, 'RecordEnd'] = pd.to_datetime(df_cleaned['RecordEnd'], errors='coerce')

        cleaned_dict[key] = df_cleaned

    return cleaned_dict

## beMTPL

In [6]:
raw_data_path = ['../raw_data/beMTPL97.rda']
df = load_rda(raw_data_path)
df_1=clean_data(df)
df_beMTPL=df_1['beMTPL97']
df_beMTPL

,id,expo,claim,nclaims,amount,average,coverage,ageph,sex,bm,power,agec,fuel,use,fleet,postcode,long,lat
0,1,1.000000,1,1,1618.001036,1618.001036,TPL,50,male,5,77,12,gasoline,private,0,1000,4.355223,50.845386
1,2,1.000000,0,0,0.000000,NaN,TPL+,64,female,5,66,3,gasoline,private,0,1000,4.355223,50.845386
2,3,1.000000,0,0,0.000000,NaN,TPL,60,male,0,70,10,diesel,private,0,1000,4.355223,50.845386
3,4,1.000000,0,0,0.000000,NaN,TPL,77,male,0,57,15,gasoline,private,0,1000,4.355223,50.845386
4,5,0.046575,1,1,155.974606,155.974606,TPL,28,female,9,70,7,gasoline,private,0,1000,4.355223,50.845386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163207,163208,1.000000,0,0,0.000000,NaN,TPL,37,male,5,40,10,diesel,work,0,9990,3.421256,51.199975
163208,163209,1.000000,0,0,0.000000,NaN,TPL,44,male,0,55,6,diesel,private,0,9990,3.421256,51.199975
163209,163210,1.000000,0,0,0.000000,NaN,TPL,50,male,0,40,10,diesel,private,0,9990,3.421256,51.199975
163210,163211,1.000000,0,0,0.000000,NaN,TPL,43,male,0,66,7,gasoline,private,0,9990,3.421256,51.199975


In [7]:
df_belge=pd.read_csv('code-postaux-belge.csv', sep=';')

In [8]:
df_code_postaux_belge=df_belge[['Code','Localite']]
df_code_postaux_belge

,Code,Localite
0,1000,Bruxelles
1,1020,Laeken
2,1040,Etterbeek
3,1070,Anderlecht
4,1140,Evere
...,...,...
2756,9880,Aalter
2757,9890,Baaigem
2758,9940,Ertvelde
2759,9968,Bassevelde


In [9]:
df_code_postaux_belge = df_code_postaux_belge.rename(columns={'Code': 'postcode'})

df_code_postaux_belge_clean = df_code_postaux_belge.drop_duplicates(subset='postcode', keep='first')

df_beMTPL = df_beMTPL.merge(
    df_code_postaux_belge_clean,
    on='postcode',
    how='left'
)


In [18]:
df_beMTPL

,id,expo,claim,nclaims,amount,average,coverage,ageph,sex,bm,power,agec,fuel,use,fleet,postcode,long,lat,Localite
0,1,1.000000,1,1,1618.001036,1618.001036,TPL,50,male,5,77,12,gasoline,private,0,1000,4.355223,50.845386,Bruxelles
1,2,1.000000,0,0,0.000000,NaN,TPL+,64,female,5,66,3,gasoline,private,0,1000,4.355223,50.845386,Bruxelles
2,3,1.000000,0,0,0.000000,NaN,TPL,60,male,0,70,10,diesel,private,0,1000,4.355223,50.845386,Bruxelles
3,4,1.000000,0,0,0.000000,NaN,TPL,77,male,0,57,15,gasoline,private,0,1000,4.355223,50.845386,Bruxelles
4,5,0.046575,1,1,155.974606,155.974606,TPL,28,female,9,70,7,gasoline,private,0,1000,4.355223,50.845386,Bruxelles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163207,163208,1.000000,0,0,0.000000,NaN,TPL,37,male,5,40,10,diesel,work,0,9990,3.421256,51.199975,Maldegem
163208,163209,1.000000,0,0,0.000000,NaN,TPL,44,male,0,55,6,diesel,private,0,9990,3.421256,51.199975,Maldegem
163209,163210,1.000000,0,0,0.000000,NaN,TPL,50,male,0,40,10,diesel,private,0,9990,3.421256,51.199975,Maldegem
163210,163211,1.000000,0,0,0.000000,NaN,TPL,43,male,0,66,7,gasoline,private,0,9990,3.421256,51.199975,Maldegem


In [20]:
url = "https://fr.wikipedia.org/wiki/Liste_des_communes_de_Belgique_par_population"

tables = pd.read_html(url)
df_densite = tables[0][['Commune', 'Hab. par km²']]

# Nettoyer les valeurs de densité : retirer \xa0 et les espaces
df_densite['Hab. par km²'] = df_densite['Hab. par km²'].astype(str).str.replace('\xa0', '', regex=False).str.replace(' ', '', regex=False).astype(int)
df_densite.to_csv('wiki_scraping.csv')
# Afficher les valeurs uniques
print(df_densite["Hab. par km²"].unique())


URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:

# --- 1. Préparer le DataFrame wiki avec densité ---
df_densite['Commune'] = df_densite['Commune'].str.replace(r'\*\*?', '', regex=True).str.strip().str.lower()
df_densite = df_densite.drop_duplicates(subset='Commune', keep='first')

# --- 2. Nettoyer les noms de commune dans df_beMTPL ---
df_beMTPL['Localite'] = df_beMTPL['Localite'].astype(str).str.strip().str.lower()

# --- 3. Créer un mapping flou des noms ---
communes_wiki = df_densite['Commune'].tolist()
unique_communes = df_beMTPL['Localite'].dropna().unique()

fuzzy_map = {
    c: process.extractOne(c, communes_wiki, scorer=fuzz.token_sort_ratio)[0]
    for c in unique_communes
}

# --- 4. Appliquer le mapping et fusionner ---
df_beMTPL['commune_match'] = df_beMTPL['Localite'].map(fuzzy_map)

df_beMTPL = df_beMTPL.merge(
    df_densite.rename(columns={'Commune': 'commune_match', 'Hab. par km²': 'hab_par_km2'}),
    on='commune_match',
    how='left'
)

NameError: name 'df_densite' is not defined

In [ ]:
df_beMTPL

NameError: name 'df_beMTPL' is not defined

In [15]:
df_beMTPL=df_beMTPL[["id","amount","nclaims","expo","power", "ageph","fuel"]]

In [16]:
df_beMTPL

,id,expo,claim,nclaims,amount,average,coverage,ageph,sex,bm,power,agec,fuel,use,hab_par_km2
0,1,1.000000,1,1,1618.001036,1618.001036,TPL,50,male,5,77,12,gasoline,private,13928
1,2,1.000000,0,0,0.000000,NaN,TPL+,64,female,5,66,3,gasoline,private,13928
2,3,1.000000,0,0,0.000000,NaN,TPL,60,male,0,70,10,diesel,private,13928
3,4,1.000000,0,0,0.000000,NaN,TPL,77,male,0,57,15,gasoline,private,13928
4,5,0.046575,1,1,155.974606,155.974606,TPL,28,female,9,70,7,gasoline,private,13928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163207,163208,1.000000,0,0,0.000000,NaN,TPL,37,male,5,40,10,diesel,work,259
163208,163209,1.000000,0,0,0.000000,NaN,TPL,44,male,0,55,6,diesel,private,259
163209,163210,1.000000,0,0,0.000000,NaN,TPL,50,male,0,40,10,diesel,private,259
163210,163211,1.000000,0,0,0.000000,NaN,TPL,43,male,0,66,7,gasoline,private,259


In [ ]:
df_beMTPL = df_beMTPL.rename(columns={
    'id': 'Id_policy',
    'amount': 'ClaimAmount',
    'nclaims': 'ClaimNb',
    'expo': 'Exposure',
    'power': 'Power',
    'ageph': 'DriverAge',
    'fuel': 'Fuel_type'
})


### Comparaison et fusion des bases euMTPL,beMTPL et freMTPL

In [6]:
print(df_freMTPL.columns)
print(df_beMTPL.columns)
print(df_euMTPL.columns)



NameError: name 'df_freMTPL' is not defined

In [7]:
#RENOMMER LES NOMS DES VARIABLES

df_freMTPL = df_freMTPL.rename(columns={
    'IdPolicy': 'Id_policy',
    'ClaimAmount': 'ClaimAmount',
    'ClaimNb': 'ClaimNb',
    'Exposure': 'Exposure',
    'power': 'Power',
    'DriverAge': 'DriverAge',
    'CarGas': 'Fuel_type'
})

df_beMTPL = df_beMTPL.rename(columns={
    'id': 'Id_policy',
    'amount': 'ClaimAmount',
    'nclaims': 'ClaimNb',
    'expo': 'Exposure',
    'power': 'Power',
    'ageph': 'DriverAge',
    'fuel': 'Fuel_type'
})

df_euMTPL = df_euMTPL.rename(columns={
    'policy_id': 'Id_policy',
    'ClaimAmount': 'ClaimAmount',
    'ClaimNb': 'ClaimNb',
    'exposure': 'Exposure',
    'horsepower': 'Power',
    'age': 'DriverAge',
    'fuel_type': 'Fuel_type'
})

# Affichage des premières lignes pour vérifier
df_euMTPL
df_freMTPL
df_beMTPL

NameError: name 'df_freMTPL' is not defined

In [8]:
# Création de la variable Sinistre dans chaque base
df_freMTPL['Sinistre'] = (df_freMTPL['ClaimAmount'] > 0).astype(int)
df_beMTPL['Sinistre'] = (df_beMTPL['ClaimAmount'] > 0).astype(int)
df_euMTPL['Sinistre'] = (df_euMTPL['ClaimAmount'] > 0).astype(int)

NameError: name 'df_freMTPL' is not defined

In [9]:
df_euMTPL['Fuel_type'].unique()

NameError: name 'df_euMTPL' is not defined

In [ ]:
# Dictionnaire pour la conversion des carburants en catégories
carburant_dict = {
    'B': 'Regular',   # Bioéthanol ou autres carburants bio
    'E': 'Regular',   # Ethanol
    'S': 'Regular',   # Super essence
    'T': 'Regular',   # Autre carburant de type essence
    'D': 'Diesel',    # Diesel
    'G': 'Diesel',    # Gaz naturel comprimé (GNC)
    'M': 'Diesel',    # Méthane (utilisé dans les véhicules au gaz)
    'P': 'Diesel'     # Propane (utilisé pour des véhicules à gaz)
}

# Appliquer la catégorisation au dataframe df_euMTPL
df_euMTPL['Fuel_type'] = df_euMTPL['Fuel_type'].map(carburant_dict)
df_euMTPL


In [ ]:
df_beMTPL['Fuel_type'].unique()

In [ ]:
df_freMTPL['Fuel_type'].unique()

In [ ]:
#Catégorisation
# Dictionnaire pour catégoriser Diesel (1) et Regular (0) dans df_beMTPL
df_beMTPL['Fuel_type'] = df_beMTPL['Fuel_type'].apply(lambda x: 1 if x == 'Diesel' else 0)
df_euMTPL['Fuel_type'] = df_euMTPL['Fuel_type'].apply(lambda x: 1 if x == 'Diesel' else 0)
df_freMTPL['Fuel_type'] = df_freMTPL['Fuel_type'].apply(lambda x: 1 if x == 'Diesel' else 0)

In [ ]:
df_euMTPL
df_beMTPL
df_freMTPL

In [1]:
# Identifier les variables communes aux trois bases
common_columns = ['Id_policy', 'ClaimAmount', 'ClaimNb', 'Exposure', 'Power', 'DriverAge', 'Fuel_type', 'Sinistre']

# Sélectionner uniquement les colonnes communes dans chaque base
df_euMTPL_common = df_euMTPL[common_columns]
df_beMTPL_common = df_beMTPL[common_columns]
df_freMTPL_common = df_freMTPL[common_columns]

# Ajouter une colonne 'Dataset' pour indiquer la provenance de chaque ligne
df_euMTPL_common['Dataset'] = 'european'
df_beMTPL_common['Dataset'] = 'belgium'
df_freMTPL_common['Dataset'] = 'french'

# Fusionner les bases en une seule base
Data = pd.concat([df_euMTPL_common, df_beMTPL_common, df_freMTPL_common], ignore_index=True)
Data



NameError: name 'df_euMTPL' is not defined